In [105]:
import pandas as pd

df = pd.read_csv("final_data.csv", sep="\t")

In [106]:
df = df.drop(['Links'], axis=1)
df = df.drop_duplicates()

In [107]:
import re
template = r'Nhà|House|Đường|Mặt\wtiền|'
template2 = r'Google|Facebook|Cửa\wgỗ|Cửa\wnhựa|Cửa\wsắt|Ad|Max|Marketing|Email|SMS'
# for tile in df['Title']:
    # check = re.search(template,tile)
    # 
# 710, 2093, 2480, 2587
hold = pd.DataFrame(columns=df.columns, data = [df.loc[710], df.loc[2093], df.loc[2480], df.loc[2587]])

df = df[~df['Title'].str.contains(template2,case=False,regex=True)]

df = pd.concat([df, hold], axis = 0)

template3 = r'tỷ'
template4 = r'\n'
df = df[df['Price'].str.contains(template3,case=False,regex=True)]
df = df[~df['Price'].str.contains(template4,case=False,regex=True)]


df["Price"] = df["Price"].apply(lambda x : x[:-3])

In [4]:
from features_prompt import extract_features, extract_title

extend_frame = pd.DataFrame(columns=["Chỗ để xe hơi", "Đang cho thuê", "CSVC xung quanh", "Vấn đề pháp lý, sổ đỏ", "Số PN", "Số WC", "ExtractedTitle"])
error_index = []
titles = []

for i in range(0, 4):
    print(i, end = "\t")
    features = extract_features(df["Description"][i], title = df["Title"][i])
    try:
        extend_frame.loc[i] = features
        print("Complete")
    except Exception as e:
        print(e)
        error_index.append(i)
    # print(df["Description"][i])

# Quay lại xử lý các index bị lỗi
while(len(error_index) > 0):
    print(error_index)
    for i in error_index:
        print(i, end = "\t")
        ans = extract_features(df["Description"][i], title = df["Title"][i])
        try:
            extend_frame.loc[i] = ans
            print("Complete")
            error_index.remove(i)
        except Exception as e:
            print(e)
    
print("Extract complete")


0	cannot set a row with mismatched columns
1	Complete
2	Complete
3	Complete
[0]
0	Complete
Extract complete


In [5]:
# nối dataframe ban đầu với các features mới
df = pd.concat([df, extend_frame], axis=1) 

# combine các cột "Số phòng ngủ" và "Số phòng WC" với nhau
df["Số phòng ngủ"] = df["Số phòng ngủ"].apply(lambda x: 0 if x == "na" else x) # với giá trị là na thay bằng 0
df["Số phòng WC"] = df["Số phòng WC"].apply(lambda x: 0 if x == "na" else x)

df["Số phòng ngủ"] = df["Số phòng ngủ"].astype(float)
df["Số phòng WC"] = df["Số phòng WC"].astype(float)

def max_two_columns_PN(row):
    return max(row['Số phòng ngủ'], row['Số PN'])
def max_two_columns_WC(row):
    return max(row['Số phòng WC'], row['Số WC'])

df['Số phòng ngủ'] = df.apply(max_two_columns_PN, axis=1)
df['Số phòng WC'] = df.apply(max_two_columns_WC, axis=1)

df = df.drop(["Số PN", "Số WC"], axis = 1)

In [6]:
df.head()

,Title,Price,Links,Diện tích (m2),Số phòng ngủ,Số phòng WC,Địa chỉ,Description,Chỗ để xe hơi,Đang cho thuê,CSVC xung quanh,"Vấn đề pháp lý, sổ đỏ",ExtractedTitle
0,(PHA23) BÁN NHÀ MẶT TIỀN ĐƯỜNG ĐỒNG NAI - PHƯỚ...,Giá bán\nDưới 500 triệu\nTừ 500 - 800 triệu\nT...,https://batdongsan.vn/pha23-ban-nha-mat-tien-d...,83,4.0,3.0,"ĐÔNG NAI, PHƯỜNG PHƯỚC HẢI, NHA TRANG",(PHA23) BÁN NHÀ MẶT TIỀN ĐƯỜNG ĐỒNG NAI - PHƯỚ...,True,False,True,False,"đường Đồng Nai, Phước Hải"
1,"Bán nhà đẹp 4 tầng Bùi Tá Hán, An Phú, Q2 4x20...",6.5 tỷ,https://batdongsan.vn/ban-nha-dep-4-tang-bui-t...,80,4.0,4.0,"Quận 2, Hồ Chí Minh","- Bán nhà mặt tiền Bùi Tá Hán, An Phú, Q2\n- D...",False,False,True,False,"Bùi Tá Hán, An Phú, Quận 2"
2,(VH25) BÁN NHÀ 3 TẦNG MẶT TIỀN ĐƯỜNG 19/5 KHU ...,15.8 tỷ,https://batdongsan.vn/vh25-ban-nha-3-tang-mat-...,138,5.0,6.0,"ĐƯỜNG 19 THÁNG 5, KHU ĐÔ THỊ VĨNH ĐIỀM TRUN...",(VH25) BÁN NHÀ 3 TẦNG MẶT TIỀN ĐƯỜNG 19/5 KHU ...,False,False,True,False,"Đường 19/5, Khu đô thị Vĩnh Điềm Trung"
3,(VH25) BÁN NHÀ 3 TẦNG MẶT TIỀN ĐƯỜNG 19/5 KHU ...,9.2 tỷ,https://batdongsan.vn/vh25-ban-nha-3-tang-mat-...,138,5.0,6.0,"ĐƯỜNG 19 THÁNG 5, KHU ĐÔ THỊ VĨNH ĐIỀM TRUN...",(VH25) BÁN NHÀ 3 TẦNG MẶT TIỀN ĐƯỜNG 19/5 KHU ...,False,True,True,False,"đường 19/5, Vịnh Điểm Trung"
4,"HXH-NGUYỄN HỮU TIẾN, TÂN PHÚ-57M2 ngang 4.1m- ...",9.2 tỷ,https://batdongsan.vn/hxh-nguyen-huu-tien-tan-...,57,4.0,3.0,na,- Vị trí trung tâm Tân Phú gần trường Đại học ...,NaN,NaN,NaN,NaN,NaN


In [4]:

df.to_csv("100PlusInstance_updated.csv", sep="\t", index=False)

In [5]:
def check_instance(index):
    print(df["Description"][index])
    print()
    print("Tiêu đề: " + df["Title"][index])
    print("\n")
    print(df.iloc[index])

check_instance(4)

- Diện tích 96m2 ngang 4.3m dài 22.5m .
- sổ vuông vức, thuận tiện xây mới, phù hợp xay dựng căn hộ dịch vụ với doanh thu cao vì gần khu dân cư, trường đại học các cấp...
- Kết cấu nhà 1 trệt, 1 lửng thông suốt , đang làm kho chứa vải
- Vị tri cách mặt tiền chỉ 30m, hẻm rộng sạch sẽ, xe tải nhỏ vào được
- Pháp lý rõ ràng, hoàn công đủ , công chứng ngay .
☎️ Lh: 0898992121 TRANG để xem nhà trực tiếp

Tiêu đề: LŨY BÁN BÍCH,TÂN PHÚ-DIỆN TÍCH KHỦNG 96M2 ( 4.3*22.5M)???? CHỈ 5TỶX


Title                    LŨY BÁN BÍCH,TÂN PHÚ-DIỆN TÍCH KHỦNG 96M2 ( 4....
Price                                                              7.35 tỷ
Links                    https://batdongsan.vn/luy-ban-bichtan-phu-dien...
Diện tích (m2)                                                          96
Số phòng ngủ                                                           2.0
Số phòng WC                                                            1.0
Location                                                            

In [6]:
extend_frame.loc[4]

Chỗ để xe hơi                                 False
Đang cho thuê                                 False
CSVC xung quanh                               False
Vấn đề pháp lý, sổ đỏ                         False
Số PN                                             0
Số WC                                             0
ExtractedTitle           Lũy Bãn Bích, Quận Tân Phú
Name: 4, dtype: object

In [8]:
df.head()

,Title,Price,Links,Diện tích (m2),Số phòng ngủ,Số phòng WC,Location,Description,Chỗ để xe hơi,Đang cho thuê,CSVC xung quanh,"Vấn đề pháp lý, sổ đỏ",ExtractedTitle
0,Chưa tới 30tr/m2 - Hàng ngộp bank BAO ĐẦU TƯ ...,Giá bán\nDưới 500 triệu\nTừ 500 - 800 triệu\nT...,https://batdongsan.vn/chua-toi-30trm2-hang-ngo...,150,2.0,1.0,"('10.7763897', '106.7011391')",Mô tả:\n+ Thông số cực đẹp 5 x 30m (150m2)\...,True,True,True,True,na
1,"Bán nhà HXH Âu Cơ Phường 9 Tân Bình, 51m2 3 Tầ...",3899000000 tỷ,https://batdongsan.vn/ban-nha-hxh-au-co-phuong...,51,3.0,0.0,NaN,"Bán nhà HXH Âu Cơ Phường 9 Tân Bình, 51m2 3 Tầ...",False,True,True,False,"đường Âu Cơ, Phường 9, Quận Tân Bình"
2,"SÁT MẶT TIỀN PHAN ĐĂNG LƯU, PHƯỜNG 7, PHÚ NHUẬ...",5.5 tỷ,https://batdongsan.vn/sat-mat-tien-phan-dang-l...,45,2.0,2.0,NaN,"- Kết cấu: 1 trệt 1 lầu BTCT; 2 PN, 2 WC. Phòn...",False,True,True,False,"Phan Đăng Lưu, Phường 7, Quận Phú Nhuận"
3,CHỦ GẤP BÁN TRƯỚC TẾT LÊ HỒNG PHONG QUẬN 5 RA ...,4.6 tỷ,https://batdongsan.vn/chu-gap-ban-truoc-tet-le...,41,2.0,0.0,"('10.7559877', '106.6785237')","Mô tả:\n- Diện tích công nhận 41m, thực tế 66m...",True,True,True,False,"đường Lê Hồng Phong, Quận 5"
4,"LŨY BÁN BÍCH,TÂN PHÚ-DIỆN TÍCH KHỦNG 96M2 ( 4....",7.35 tỷ,https://batdongsan.vn/luy-ban-bichtan-phu-dien...,96,2.0,1.0,NaN,- Diện tích 96m2 ngang 4.3m dài 22.5m .\n- sổ ...,False,False,False,False,"Lũy Bãn Bích, Quận Tân Phú"
